# 本章简介
> 本章介绍了集成学习技术的实现和使用方法：多数投票、bagging、boosting

在本章我们将构建构建一组分类器的集合，使得整体分类效果优于任意一个单独的分类器  
+ 基于多数投票的预测
+ 通过对训练数据集的重复抽样和随机组合降低模型的过拟合
+ 通过弱学习机在误分类数据上的学习构建性能更好的模型 5  

# 7.1 集成学习
> 介绍了集成学习的实现原理-通过不同的分类器组成一个元分类器

集成方法的目标是：将不同的分类器组合成为一个元分类器，与包含于其中的单个分类器相比，元分类器具有更好的泛化性能  
本章介绍的集成方法都试用了多数投票原则：二类别分类情形下将得票率超过50%的结果作为类标  
下图介绍了集成10个分类器时，多数及简单多数票法表决的概念，其中每个单独的符号分别代表一个类标  
![7-1 5](../syn_pic/py_machine_learning/7-1.png)
基于训练集，首先训练m个不同的成员分类器，在多数投票原则下，可集成不同的分类算法，也可使用相同的成员分类器拟合不同的训练子集  
下图介绍了使用多数投票原则的通用集成方法的概念  
![7-2](../syn_pic/py_machine_learning/7-2.png)
我们汇总所有分类器$C_j$的预测类标，选出得票率最高的类标$\hat{y}$
$$\hat{y}=mode\{C_1(x),C_2(x),\dots,C_m(x)\}$$
5  
在二类别分类中，假定class1=-1、class2=+1，我们可以将多数投票预测表示为：  
$$C(x)=sign\left[\sum_j^m{C_j(x)}\right]=\begin{cases} 1&若\sum_i{C_j(x)\ge0} \cr -1&其他 \end{cases}$$
我们假定二类别分类中的n个成员分类器都有相同的出错率$\varepsilon$，假定每个分类器都是独立的，出错率之间是不相关的  
基于这些假设，我们可以将成员分类器集成后的出错概率简单地表示为二项分布的概率密度函数 5  
$$P(y\ge k)=\sum_k^n{\left\langle\begin{matrix}n \cr k\end{matrix}\right\rangle}\varepsilon^k(1-\varepsilon)^{n-k}=\varepsilon_{ensemble}$$
其中，$\left\langle\begin{matrix}n \cr k\end{matrix}\right\rangle$是n选k组合的二项式系数  
再来看一个更具体的例子：假定使用11个分类器(n=11)，单个分类器出错率为0.25  
$$P(y\ge k)=\sum_{k=6}^n{\left\langle\begin{matrix}11 \cr k\end{matrix}\right\rangle}0.25^k(1-\varepsilon)^{11-k}=0.034$$
5  
在满足所有假设的条件下，集成后的出错率远小于单个分类器的出错率  
我们用户Python比较一下成员分类器在不同出错率情况下与集成分类器出错率的差异  

In [ ]:
'''
python	scipy	Special functions (scipy.special)	comb()
python	math — Mathematical functions	Number-theoretic and representation functions	ceil()
python	Built-in Functions	sum()	sum()
'''
from scipy.special import comb
import math
def ensemble_error(n_classifier, error):
    k_start = math.ceil(n_classifier / 2.0)
    probs = [comb(n_classifier, k) * error**k *
            (1-error)**(n_classifier - k)
            for k in range(k_start, n_classifier + 1)]
    return sum(probs)
ensemble_error(n_classifier=11, error=0.25)

在成员分类器出错率介于0.0到1.0范围内，我们可以以函数曲线的形式绘制出它们之间的关系  

In [ ]:
'''
5
'''
import numpy as np
import matplotlib.pyplot as plt
error_range = np.arange(0.0, 1.01, 0.01)
ens_errors = [ensemble_error(n_classifier=11, error=error)
             for error in error_range]
plt.plot(error_range, ens_errors,
        label='Ensemble error',
        linewidth=2)
plt.plot(error_range, error_range,
        linestyle='--', label='Base error',
        linewidth=2)
plt.xlabel('Base error')
plt.ylabel('Base/Ensemble error')
plt.legend(loc='upper left')
plt.grid()
plt.show()

从结果可见：当成员分类器出错率低于随机猜测时($\varepsilon\gt0.5$)，集成分类器的出错率要低于单个分类器 5  
# 7.2 实现一个简单的多数投票分类器
> 介绍了如何实现一个多数投票分类器及其效果

我们的目标是构建一个更加强大的元分类器，以在特定的数据集上平衡单个分类器的弱点。可以将加权多数投票记为  
$$\hat{y}=arg \max \limits_i\sum_{j=1}^m{w_j\chi_A(C_j(x)=i)}$$
其中,$w_j$是成员分类器$C_j$对应的权重，$\hat{y}$为集成分类器的预测类标，$\chi_A$为特征函数$[C_j(x)=i\in A]$，A为类别的集合。如果权重相等，可简化为 5  
$$\hat{y}=mode\{C_1(x),C_2(x),\dots,C_m(x)\}$$
为了更好地理解权重的含义，我们来看几个例子。假定三个成员分类器$C_j(i\in\{1,2,3\})$,分别用它们来预测样本x的类标：  
$$C_1(x)\rightarrow0,C_2(x)\rightarrow0,C_3(x)\rightarrow1$$
$$\hat{y}=mode\{0,0,1\}=0$$
5  
我们将权重0.6赋值给$C_3$，而$C_1$和$C_2$均为0.2，有：  
$$\hat{y}=arg \max \limits_i\sum_{j=1}^m{w_j\chi_A(C_j(x)=i)}$$
$$=arg \max \limits_i\sum_{j=1}^m{w_j\chi_A(C_j(x)=i)}$$
更直观地，由于$3\times 0.2=0.6$，可以认为分类器$C_3$的一次预测权重相当于分类器$C_1$或$C_2$的三次预测权重之和，我们可以写作：  
$$\hat{y}=mode\{0,0,1,1,1\}=1$$
5  
我们可以使用numpy的argmax和bincount函数来实现加权投票  

In [ ]:
'''
python	numpy	Sorting, searching, and counting	argmax()
python	numpy	Statistics	bincount()
'''
np.argmax(np.bincount([0, 0, 1], weights=[0.2, 0.2, 0.6]))

如果集成分类器事先得到良好的修正，那么在多数投票中使用预测类别的概率来替代类标会非常有用。使用类别概率进行预测的多数投票修改版本可记为：  
$$\hat{y}=arg \max \limits_i\sum_{j=1}^m{w_jp_{ij}}$$
其中，$p_{ij}$是第j个分类器预测样本类标为i的概率 5  
假设我们面临一个二类别分类问题，其类标为$i\in\{0,1\}$，下面集成这三个分类器$C_j(j\in\{1,2,3\})$。假定分类器$C_j$针对某一样本x按照如下概率返回类标的预测结果：  
$$C_1(x)\rightarrow[0.9,0.1],C_2(x)\rightarrow[0.8,0.2],C_3(x)\rightarrow[0.4,0.6]$$
我们可以按照如下方式计算所属类别的概率：  
$$p(i_0|x)=0.2\times 0.9+0.2\times0.8+0.6\times0.4=0.58$$
$$p(i_i|x)=0.2\times 0.1+0.2\times0.2+0.6\times0.06=0.42$$
5  
$$\hat{y}=arg \max\limits_i[p(i_0|x),p(i_1|x)]=0$$
为实现基于类别预测概率的加权多数投票，我们可以再次使用Numpy中的numpy.average和np.argmax方法  

In [ ]:
'''
python	numpy	Statistics	average()b
'''
ex = np.array([[0.9, 0.1],
             [0.8, 0.2],
             [0.4, 0.6]])
p = np.average(ex, axis=0, weights=[0.2, 0.2, 0.6])
p

In [ ]:
np.argmax(p)

我们来实现MajorityVoteClassifier类 5  

In [ ]:
'''
5
'''
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import operator

class MajorityVoteClassifier(BaseEstimator,
                            ClassifierMixin):
    """ A majority vote ensemble classifier
    
    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
     Different classifiers for the ensemble
     
    vote : str, {'classlabel', 'probability'}
     Default: 'classlabel'
     If 'classlabel' the prediction is based on 
     the argmax of class labels. Else if
     'probability', the argmax of the sum of
     probabilities is used to predict the class label
     (recommended for calibrated classifiers).
     
    weights : array-like, shape = [n_classifiers]
     Optional, default: None
     If a list of `int` or `float` values are
    provided, the classifiers are weighted by
    importance; Uses uniform weights if `weights=None`.
    
    """
    def __init__(self, classifiers,
                vote='classlabel', weights=None):
        self.classifiers = classifiers
        self.named_classifiers = {key: value for
                                 key, value in 
                                 _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights
        
    def fit(self, X, y):
        """ Fit classifiers.
        
        Parameters
        ----------
        X : {array-like, sparse matix}，
            shape = [n_samples, n_features]
            Matrux of training samples.
        
        y : array-like, shape = [n_samples]
            Vector of target class lables.
            
        Returns
        -------
        self : object
        """
        # Use LabelEncoder to ensure class labels start
        # with 0, which is important for np.argmax
        # call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X,
                                       self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self